In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
file_path = 'Orders.csv'
data = pd.read_csv(file_path)
data.head()

,Order ID,Product Name,Category,Purchase Date,Amount (USD),Quantity,Payment Method,Savings (USD),Bank Name,Card Type
0,10001,iPhone 13,Electronics,2023-05-12,1065.33,1,Credit Card,40.00,HDFC Bank,Visa
1,10002,"Samsung 55"" Smart LED TV",Electronics,2023-04-23,666.65,1,Debit Card,66.67,ICICI Bank,RuPay
2,10003,Philips Air Fryer,Home & Kitchen,2023-06-15,119.99,1,Net Banking,13.33,-,-
3,10004,Adidas Running Shoes,Fashion,2023-03-29,66.65,1,Credit Card,6.67,SBI Bank,MasterCard
4,10005,The Alchemist by Paulo Coelho,Books,2023-02-19,5.32,1,UPI,0.67,-,-


In [ ]:
data['Purchase Date'] = pd.to_datetime(data['Purchase Date'])
data.head()

,Order ID,Product Name,Category,Purchase Date,Amount (USD),Quantity,Payment Method,Savings (USD),Bank Name,Card Type
0,10001,iPhone 13,Electronics,2023-05-12,1065.33,1,Credit Card,40.00,HDFC Bank,Visa
1,10002,"Samsung 55"" Smart LED TV",Electronics,2023-04-23,666.65,1,Debit Card,66.67,ICICI Bank,RuPay
2,10003,Philips Air Fryer,Home & Kitchen,2023-06-15,119.99,1,Net Banking,13.33,-,-
3,10004,Adidas Running Shoes,Fashion,2023-03-29,66.65,1,Credit Card,6.67,SBI Bank,MasterCard
4,10005,The Alchemist by Paulo Coelho,Books,2023-02-19,5.32,1,UPI,0.67,-,-


In [ ]:
data['YearMonth'] = data['Purchase Date'].dt.to_period('M')
data.head()

,Order ID,Product Name,Category,Purchase Date,Amount (USD),Quantity,Payment Method,Savings (USD),Bank Name,Card Type,YearMonth
0,10001,iPhone 13,Electronics,2023-05-12,1065.33,1,Credit Card,40.00,HDFC Bank,Visa,2023-05
1,10002,"Samsung 55"" Smart LED TV",Electronics,2023-04-23,666.65,1,Debit Card,66.67,ICICI Bank,RuPay,2023-04
2,10003,Philips Air Fryer,Home & Kitchen,2023-06-15,119.99,1,Net Banking,13.33,-,-,2023-06
3,10004,Adidas Running Shoes,Fashion,2023-03-29,66.65,1,Credit Card,6.67,SBI Bank,MasterCard,2023-03
4,10005,The Alchemist by Paulo Coelho,Books,2023-02-19,5.32,1,UPI,0.67,-,-,2023-02


In [ ]:
monthly_data = data.groupby('YearMonth').agg({'Amount (USD)': 'sum'}).reset_index()
monthly_data.head()

,YearMonth,Amount (USD)
0,2022-01,1789.96
1,2022-02,1173.27
2,2022-03,379.94
3,2022-04,1459.96
4,2022-05,797.24


In [ ]:
monthly_data['YearMonth'] = monthly_data['YearMonth'].dt.to_timestamp()
monthly_data.head()

,YearMonth,Amount (USD)
0,2022-01-01,1789.96
1,2022-02-01,1173.27
2,2022-03-01,379.94
3,2022-04-01,1459.96
4,2022-05-01,797.24


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(monthly_data[['Amount (USD)']])
scaled_data

array([[0.37768831],
       [0.21250157],
       [0.        ],
       [0.28929443],
       [0.11177808],
       [1.        ],
       [0.75713915],
       [0.01857611],
       [0.03589863],
       [0.10714946],
       [0.55001085],
       [0.24376087],
       [0.48001093],
       [0.02071631],
       [0.43677828],
       [0.3000008 ],
       [0.49695845],
       [0.38391874],
       [0.54822154]])

In [ ]:
def create_dataset(dataset, look_back):
    X, y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(y)
look_back = 6
X, y = create_dataset(scaled_data, look_back)
y

array([0.75713915, 0.01857611, 0.03589863, 0.10714946, 0.55001085,
       0.24376087, 0.48001093, 0.02071631, 0.43677828, 0.3000008 ,
       0.49695845, 0.38391874])

In [ ]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
X

array([[[0.37768831],
        [0.21250157],
        [0.        ],
        [0.28929443],
        [0.11177808],
        [1.        ]],

       [[0.21250157],
        [0.        ],
        [0.28929443],
        [0.11177808],
        [1.        ],
        [0.75713915]],

       [[0.        ],
        [0.28929443],
        [0.11177808],
        [1.        ],
        [0.75713915],
        [0.01857611]],

       [[0.28929443],
        [0.11177808],
        [1.        ],
        [0.75713915],
        [0.01857611],
        [0.03589863]],

       [[0.11177808],
        [1.        ],
        [0.75713915],
        [0.01857611],
        [0.03589863],
        [0.10714946]],

       [[1.        ],
        [0.75713915],
        [0.01857611],
        [0.03589863],
        [0.10714946],
        [0.55001085]],

       [[0.75713915],
        [0.01857611],
        [0.03589863],
        [0.10714946],
        [0.55001085],
        [0.24376087]],

       [[0.01857611],
        [0.03589863],
        [0.1071494

In [ ]:
train_size = int(len(X) * 0.80)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
y_train, y_test = y[0:train_size], y[train_size:len(y)]
y_train

array([0.75713915, 0.01857611, 0.03589863, 0.10714946, 0.55001085,
       0.24376087, 0.48001093, 0.02071631, 0.43677828])

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=2)

Epoch 1/20


C:\Users\mswat\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 - 3s - 359ms/step - loss: 0.1083
Epoch 2/20
9/9 - 0s - 19ms/step - loss: 0.0684
Epoch 3/20
9/9 - 0s - 15ms/step - loss: 0.0723
Epoch 4/20
9/9 - 0s - 7ms/step - loss: 0.0722
Epoch 5/20
9/9 - 0s - 7ms/step - loss: 0.0695
Epoch 6/20
9/9 - 0s - 7ms/step - loss: 0.0703
Epoch 7/20
9/9 - 0s - 9ms/step - loss: 0.0679
Epoch 8/20
9/9 - 0s - 8ms/step - loss: 0.0764
Epoch 9/20
9/9 - 0s - 7ms/step - loss: 0.0721
Epoch 10/20
9/9 - 0s - 10ms/step - loss: 0.0704
Epoch 11/20
9/9 - 0s - 7ms/step - loss: 0.0669
Epoch 12/20
9/9 - 0s - 7ms/step - loss: 0.0690
Epoch 13/20
9/9 - 0s - 8ms/step - loss: 0.0687
Epoch 14/20
9/9 - 0s - 7ms/step - loss: 0.0692
Epoch 15/20
9/9 - 0s - 6ms/step - loss: 0.0663
Epoch 16/20
9/9 - 0s - 8ms/step - loss: 0.0666
Epoch 17/20
9/9 - 0s - 5ms/step - loss: 0.0682
Epoch 18/20
9/9 - 0s - 8ms/step - loss: 0.0654
Epoch 19/20
9/9 - 0s - 6ms/step - loss: 0.0669
Epoch 20/20
9/9 - 0s - 6ms/step - loss: 0.0722


In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
train_predict

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[0.22368713],
       [0.20152265],
       [0.21026672],
       [0.24712962],
       [0.2666954 ],
       [0.29024446],
       [0.24574012],
       [0.20015825],
       [0.21769385]], dtype=float32)

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train = scaler.inverse_transform([y_train])
y_test = scaler.inverse_transform([y_test])
y_train

array([[3206.56,  449.29,  513.96,  779.96, 2433.29, 1289.97, 2171.96,
         457.28, 2010.56]])

In [ ]:
train_predict

array([[1215.0289],
       [1132.2825],
       [1164.9266],
       [1302.5465],
       [1375.5913],
       [1463.5067],
       [1297.3591],
       [1127.1888],
       [1192.6543]], dtype=float32)

In [ ]:
future_predictions = []
last_lags = X_test[-1].reshape(1, look_back, 1)
last_lags

array([[[0.24376087],
        [0.48001093],
        [0.02071631],
        [0.43677828],
        [0.3000008 ],
        [0.49695845]]])

In [ ]:
for _ in range(4):
    future_pred = model.predict(last_lags)
    future_predictions.append(scaler.inverse_transform(future_pred)[0][0])
    
    last_lags = np.roll(last_lags, -1)
    last_lags[0, -1] = future_pred


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [ ]:
future_dates = pd.date_range(start=monthly_data['YearMonth'].iloc[-1], periods=5, freq='M')[1:]
future_data = pd.DataFrame({'YearMonth': future_dates, 'Amount (USD)': future_predictions})

C:\Users\mswat\AppData\Local\Temp\ipykernel_1156\3318211398.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start=monthly_data['YearMonth'].iloc[-1], periods=5, freq='M')[1:]


In [ ]:
last_actual_value = monthly_data[['YearMonth', 'Amount (USD)']].iloc[-1:]
combined_data = pd.concat([monthly_data[['YearMonth', 'Amount (USD)']], future_data])
combined_data_with_last_actual = pd.concat([last_actual_value, future_data])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_data['YearMonth'], y=monthly_data['Amount (USD)'],
                         mode='lines+markers', name='Historical Spending'))

fig.add_trace(go.Scatter(x=combined_data_with_last_actual['YearMonth'], y=combined_data_with_last_actual['Amount (USD)'],
                         mode='lines+markers', name='Future Predictions'))

fig.update_layout(title='Customer Spending Prediction with LSTM',
                  xaxis_title='Date',
                  yaxis_title='Amount (USD)',
                  legend=dict(x=0, y=1))

fig.show()